# Multi-Point measurement

This notebook is designed to repeat a measurement at multiple user-defined positions.

The list of positions can be set by hand by moving the stage and repeatedly getting the current stage coordinates from Imspector.

In [4]:
import specpy

from autosted.callback_buildingblocks.regular_position_generators import PositionListOffsetGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.pipeline import AcquisitionPipeline
from autosted.imspector import get_current_stage_coords

## 1: Parameters

For this automation pipeline, we only need to know where to save our data and which settings to use for each position

In [1]:
# where to save & whether to save combined HDF5 file
save_folder = 'examples/acquisition_data/multipoint'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'examples/config_json/20241010_overview_3d_640.json'

### 1a (OPTIONAL Alternative): Use settings from current measurement in Imspector

Run this cell to re-use the settings of current measurement (and configuration) in Imspector instead of loading them from a file.

In [ ]:
imspector = specpy.get_application()
measurement_parameters = imspector.active_measurement().parameters('')

### 1b (OPTIONAL Alternative): Use settings from current measurement in Imspector (multiple configurations)

Run this cell to re-use the settings of current measurement in Imspector.

In contrast to 1a, this will use all configurations of the current measurement and run them all at each position in the multipoint experiment. 

In [ ]:
measurement_parameters = []

measurement = imspector.active_measurement()
for configuration in measurement.configuration_names():
    measurement.activate(measurement.configuration(configuration))
    measurement_parameters.append(measurement.parameters(''))

## 2: Building list of coordinates to image

Now, we manually build a list of stage coordinates to image.

Run the second cell multiple times (after moving to positions you want to image at) to add multiple points.

In [2]:
# start with empty position list
coordinate_list = []

In [7]:
# append current stage position, run multiple times to enqueue multiple positions to image at
coordinate_list.append(get_current_stage_coords())

In [ ]:
# optionally reverse position list (saves initial move if we start right after setting last position)
coordinate_list = coordinate_list[::-1]

In [ ]:
# show for verification
coordinate_list

## 3: run multipoint acquisition

In [9]:
# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('position',
    JSONSettingsLoader(measurement_parameters),
    PositionListOffsetGenerator(coordinate_list, return_parameter_dict=True))

# build pipeline object (1 level: 'position')
pipeline = AcquisitionPipeline(save_folder, ['position'], save_combined_hdf5=save_hdf5)
# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'position')

# start with initial task from callback
pipeline.run(next_position_generator)